# Generate communication network


This notebook contains the workflow to simplify the technical knudepunkter network (raw input data from GeoFA) into the "communication" network where we only have 1 node at each intersection, and only 1 edge at each street (i.e. no parallel lines even when network runs on both sides of a street, etc.)

Final output: a gdf which can be directly converted into a simplfied network by `momepy.gdf_to_nx` (see notebook [`read-network.ipynb`](./read-network.ipynb) for sample code)

> Caveat: the voronoi skeletonization takes several hours to run for the largest network components

* ~~install sgeop bleeding edge~~
* ~~import all raw edges~~
* ~~to proj crs~~
* ~~buffer around all edges (custom, now: 25m)~~
* ~~union of buffer polygons~~
* ~~get boundary and explode it~~
* ~~drop too-short linestrings (assuming loops within)~~
* ~~polygonize~~
* ~~only keep those polygons that have interiors~~
* ~~now we have one polygon per network component~~
* ~~for each polygon:~~
	* get delineation (all interiors and the exterior)
	* unzip delineation lines, with modulo caveat not to lose geoms
	* skeletonize (delineation lines and comp poly) with sgeop.geometry.voronoi_skeleton
	* convert to nx
	* should be only 1 component!! (#TODO : consolidate nodes with sgeop.nodes.consolidate_nodes OR shapely.snap?) 
	* iteratively remove degree-one nodes if dangling edge is short enough
	* remove false nodes
* ~~combine into one geodataframe~~
* ~~save~~ 

**todo: check whether we can get data from [here](https://geofa-kort.geodanmark.dk/app/fkg/?config=/api/v2/configuration/fkg/configuration_fkg_udgivet_5f465f5d3181f687353260.json#Basis_kort/11/12.0939/55.2226/fkg.t_5609_cykelknudepunktsstraekninger,fkg.t_5608_cykelknudepunkter) and what the difference is**

(https://geofa.geodanmark.dk/ows/fkg/fkg)

if so: start with `./src/wfs-func.py`, we already have some functions defined there for WFS import

In [136]:
# import libraries
import os
import osmnx as ox
import pandas as pd
import geopandas as gpd
import folium
import momepy
import networkx as nx
import matplotlib.pyplot as plt
import shapely
from shapely.geometry import LineString
import numpy as np
from sgeop.geometry import voronoi_skeleton
from src.nw import unzip_line, drop_dangling_edges_iter
import time

make folders for output

In [ ]:
os.makedirs("../data/network-communication/", exist_ok=True)
os.makedirs("../data/network-communication/raw/", exist_ok=True)

define parameters

In [ ]:
# define parameters
proj_crs = 'EPSG:25832' # in meters
my_buffer = 25 # in meters: buffer around edges for skeletonization
my_length_threshold = 500 # in meters: buffered edge polygon boundaries below will be dropped (assuming interior, not actual loop)
my_maxseglen = 30 # in meters: for voronoi skeletonization
my_danglefactor = 2.5 # _dangling_ edges of length below my_buffer * my_danglefactor will be considered too short to keep in NW

load input data

In [ ]:
# load "technical" files
nodes = gpd.read_file("../data/network-technical/nodes.gpkg")
nodes = nodes.to_crs(proj_crs)
edges = gpd.read_file("../data/network-technical/edges.gpkg")
edges = edges.to_crs(proj_crs)

In [ ]:
## map to check data
# m=edges[["geometry"]].explore(tiles = "cartodb positron", name = "edges", color = "blue", prefer_canvas=True)
# nodes[["geometry"]].explore(m=m,name="nodes", color = "green")
# folium.LayerControl().add_to(m)
# m

for generating a connected network, first: get boundaries of polygon buffers around edges

In [ ]:
# "bou" is set of boundaries of polygons from buffered edges
geom = edges.buffer(my_buffer).union_all()
ser = gpd.GeoSeries([geom], crs = edges.crs)
bou = ser.boundary
bou = bou.explode().reset_index(drop=True)
bou = gpd.GeoDataFrame({"geometry":bou}, crs = edges.crs)
bou["length"] = bou.length

In [ ]:
## map to check which boundaries will be dropped
# m=bou[bou.length<=my_length_threshold].explore(tiles = "cartodb positron", color = "red", style_kwds={"weight":4}, name="drop")
# edges.explore(m=m, color = "black", name = "edges", style_kwds={"weight":1}) 
# nodes.explore(m=m, color = "blue", name = "nodes")
# folium.LayerControl().add_to(m)
# m

drop boundaries below length threshold

In [ ]:
# drop too-short linestrings
bou_red = bou[bou.length>my_length_threshold].copy()

polygonize linestrings as input for skeletonization

In [ ]:
# polygonize
poly = shapely.polygonize(
    np.array(
        bou_red.geometry
    )
)

gdf = gpd.GeoDataFrame(
    {
        "geometry": poly.geoms
    },
    crs = edges.crs
)

# only keep those polygons that have interiors
gdf = gdf[[i!=[] for i in gdf.interiors]].reset_index(drop=True)

# sort by area
gdf["area"] = gdf.area
gdf = gdf.sort_values(by="area").reset_index(drop=True)
del gdf["area"]

gdf["i"] = gdf.index

In [ ]:
print(len(gdf))

**skeletonize (each component/polygon separately)**

In [ ]:
for i, comp in enumerate(gdf.geometry):

    # get comp geom
    print(f"comp {i}:", np.round(comp.area/10**6, 2), "km2")

    # get delineation
    delin = gpd.GeoDataFrame(
        {
            "geometry":
            [
                LineString(r) for r in comp.interiors] + [LineString(comp.exterior)
            ]
        },
        crs = edges.crs
    )

    # unzip lines into 2-point edges
    all_lines = []
    for geom in delin.geometry:
        all_lines+= (unzip_line(geom))
    lines = gpd.GeoDataFrame({"geometry": all_lines}, crs = edges.crs)
    lines["i"] = lines.index

    # skeletonize
    print(f"skeletonizing comp {i}...")
    t1 = time.time()
    skel = voronoi_skeleton(
        lines = lines.geometry,
        poly = comp,
        max_segment_length = my_maxseglen
    )
    t2 = time.time()
    t = np.round(t2-t1)
    print(f"finished in {t} seconds")
    skel_gdf = gpd.GeoDataFrame({"geometry":skel[0]}, crs = edges.crs)

    # dropping dangling edges from skeletonization

    # save raw skel (for merging if disconnected)
    skel_gdf[["geometry"]].to_file(f"../data/network-communication/raw/{i}_skel.gpkg", index = False)
    edges_rem = drop_dangling_edges_iter(skel_gdf, my_danglefactor, my_buffer)

    # convert to network to check component number
    G_rem = momepy.gdf_to_nx(gdf_network = edges_rem, multigraph=False, integer_labels=True)
    comps = sorted([c for c in nx.connected_components(G_rem)], key=len, reverse=True)
    print(f"Comps lens for {i}: ", [len(c) for c in comps])

    # save file
    edges_rem[["geometry"]].to_file(f"../data/network-communication/raw/{i}.gpkg", index = False)

In [ ]:
# # map to check outputs
# m=lines.explore(tiles="cartodb positron", column="i", name = "lines")
# nodes.explore(m=m, name = "nodes")
# edges.explore(m=m, name = "edges")
# skel_gdf.explore(m=m, name = "skel", color = "red")
# edges_rem.explore(m=m, name = "edges_rem", color = "orange")
# folium.LayerControl().add_to(m)
# m

combine all component gdfs into one network to save as graphml

In [126]:
gdfs = []
for i in range(len(gdf)):
    sub_gdf = gpd.read_file(f"../data/network-communication/raw/{i}.gpkg")
    sub_gdf["comp"] = i
    gdfs.append(sub_gdf)

In [127]:
gdfs = pd.concat(gdfs).reset_index(drop=True)

In [134]:
gdfs.to_file("../data/network-communication/gdfs.gpkg", index = False)